In [1]:
from aws.src.database.dynamodb.utils.get_dynamodb_table import get_dynamodb_table
from enums.enums import Stage
from aws.src.database.domain.dynamo_domain_objects import Patch

table = get_dynamodb_table("PatchesAndAreas", Stage.HOUSING_DEVELOPMENT)
patches_raw = table.scan()["Items"]
patches = [Patch.from_data(patch) for patch in patches_raw]

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.eu-west-1.amazonaws.com/

Then enter the code:

QHHB-VVTG
Successfully logged into Start URL: https://hackney.awsapps.com/start#
Stage / AWS Profile: [housing-development]
Account ID: 364864573329
Assumed Role: AWSReservedSSO_mtfh_developer_4658dde898517097
User: adam.tracy@hackney.gov.uk
Connected to DynamoDB table PatchesAndAreas


In [5]:
areas = [patch for patch in patches if patch.patchType == "area"]
patches_only = [patch for patch in patches if patch.patchType == "patch"]

unique_areas_names = set([area.name for area in areas])
unique_areas: list[Patch] = []
for name in unique_areas_names:
    area = [area for area in areas if area.name == name][0]
    unique_areas.append(area)


unique_patch_names = set([patch.name for patch in patches_only])
unique_patches: list[Patch] = []
for patch_name in unique_patch_names:
    patch = [patch for patch in patches_only if patch.name == patch_name][0]
    unique_patches.append(patch)

unique_patches.sort(key = lambda x: x.name)
unique_areas.sort(key = lambda x: x.name)

print(f"Unique Areas: {[area.name for area in unique_areas]}")
print(f"Unique Patches: {[patch.name for patch in unique_patches]}")

Unique Areas: ['CL Area', 'CP Area', 'HN1 Area', 'HN2 Area', 'SD Area', 'SH Area', 'SN Area']
Unique Patches: ['CL1', 'CL2', 'CL3', 'CL4', 'CL5', 'CL6', 'CL7', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CP7', 'HN1', 'HN10', 'HN11', 'HN2', 'HN3', 'HN4', 'HN5', 'HN6', 'HN7', 'HN8', 'HN9', 'SD1', 'SD2', 'SD3', 'SD4', 'SD5', 'SD6', 'SD7', 'SD8', 'SD9', 'SH1', 'SH2', 'SH3', 'SH4', 'SH5', 'SH6', 'SN1', 'SN2', 'SN3', 'SN4', 'SN5', 'SN7', 'SN8']


In [6]:
# Map unique patches to unique areas for each area code

# Handle HN1 and HN2 separately
hn1_area = [area for area in unique_areas if area.name == "HN1 Area"][0]
hn2_area = [area for area in unique_areas if area.name == "HN2 Area"][0]

hn1_patch_names = [f"HN{i}" for i in [2, 5, 8, 9, 10, 11]]
hn2_patch_names = [f"HN{i}" for i in [1, 3, 4, 6, 7]]

hn_patches = [patch for patch in unique_patches if "HN" in patch.name]
for patch in hn_patches:
    if patch.name in hn1_patch_names:
        patch.parentId = hn1_area.id
    elif patch.name in hn2_patch_names:
        patch.parentId = hn2_area.id
    else:
        print(f"Patch {patch.name} not found in HN1 or HN2")
        break

other_patches = [patch for patch in unique_patches if "HN" not in patch.name]
other_areas = [area for area in unique_areas if area.name not in ["HN1 Area", "HN2 Area"]]
for area in unique_areas:
    area_code = area.name.replace(" Area", "")
    area_patches = [patch for patch in other_patches if area_code == patch.name[0:2]]
    for patch in area_patches:
        patch.parentId = area.id

for area in unique_areas:
    patches_with_area = [patch.name for patch in unique_patches if patch.parentId == area.id]
    print(f"Area: {area.name} has patches {patches_with_area}")

Area: CL Area has patches ['CL1', 'CL2', 'CL3', 'CL4', 'CL5', 'CL6', 'CL7']
Area: CP Area has patches ['CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CP7']
Area: HN1 Area has patches ['HN10', 'HN11', 'HN2', 'HN5', 'HN8', 'HN9']
Area: HN2 Area has patches ['HN1', 'HN3', 'HN4', 'HN6', 'HN7']
Area: SD Area has patches ['SD1', 'SD2', 'SD3', 'SD4', 'SD5', 'SD6', 'SD7', 'SD8', 'SD9']
Area: SH Area has patches ['SH1', 'SH2', 'SH3', 'SH4', 'SH5', 'SH6']
Area: SN Area has patches ['SN1', 'SN2', 'SN3', 'SN4', 'SN5', 'SN7', 'SN8']


In [12]:
# Update the database with the parent Ids of the patches

for patch in unique_patches:
    table.update_item(
        Key={"id": patch.id},
        UpdateExpression="SET parentId = :parentId",
        ExpressionAttributeValues={":parentId": patch.parentId},
    )

# Delete all items in the table that aren't in the unique patches and unique areas
unique_patches_and_areas = unique_patches + unique_areas

all_area_ids = [area.id for area in areas]
unique_area_ids = [area.id for area in unique_patches_and_areas]

all_patch_ids = [patch.id for patch in patches]
unique_patch_ids = [patch.id for patch in unique_patches_and_areas]

area_ids_to_delete = [area_id for area_id in all_area_ids if area_id not in unique_area_ids]
patch_ids_to_delete = [patch_id for patch_id in all_patch_ids if patch_id not in unique_patch_ids]

print(f"Deleting {len(area_ids_to_delete)}/{len(all_area_ids)} areas and {len(patch_ids_to_delete)}/{len(all_patch_ids)} patches")
print(f"Keeping {len(unique_area_ids)} areas and {len(unique_patch_ids)} patches")

res = input("Confirm delete? (y/n): ")
assert res.lower() in ["y", "yes"], "Delete aborted"

for area_id in area_ids_to_delete:
    table.delete_item(Key={"id": area_id})

for patch_id in patch_ids_to_delete:
    table.delete_item(Key={"id": patch_id})

Deleting 24/31 areas and 182/236 patches
Keeping 54 areas and 54 patches


AssertionError: Delete aborted